# Görev-1

In [1]:
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)



def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [2]:
creds = {'user': 'synan_dsmlbc',
         'passwd': 'haydegidelum',
         'host': '34.91.75.155',
         'port': 3306,
         "db": "synan_dsmlbc_db"}


connstr = 'mysql+mysqlconnector://{user}:{passwd}@{host}:{port}/{db}'

conn = create_engine(connstr.format(**creds))

In [3]:
pd.read_sql_query("show databases", conn)

,Database
0,information_schema
1,synan_dsmlbc_db
2,synan_dsmlbc_group_1
3,synan_dsmlbc_group_2
4,synan_dsmlbc_group_3
5,synan_dsmlbc_group_4
6,synan_dsmlbc_group_5
7,synan_dsmlbc_group_6
8,synan_dsmlbc_group_7
9,synan_dsmlbc_group_8


In [4]:
pd.read_sql_query("show tables", conn)

,Tables_in_synan_dsmlbc_db
0,cltv_prediction
1,cltv_prediction_dsmlbc6
2,cltv_prediction_dsmlbc7
3,cltv_prediction_mvk
4,emin_serkan_erdonmez
5,emre_topalgokceli
6,onder_ekrem_tanrikulu_cltv
7,online_retail_2009_2010
8,online_retail_2010_2011
9,online_retail_all


In [5]:
retail_mysql_df = pd.read_sql_query("select * from synan_dsmlbc_group_2.online_retail_2010_2011", conn)

In [6]:
retail_mysql_df.head()

,id,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,unix
0,1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,1291191960
1,2,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,1291191960
2,3,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,1291191960
3,4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,1291191960
4,5,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,1291191960


In [7]:
df = retail_mysql_df.copy()

In [8]:
#########################
# Veri Ön İşleme
#########################

# Ön İşleme Öncesi
df.describe().T
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [9]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
today_date = dt.datetime(2011, 12, 11)

In [10]:
cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [11]:
cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

In [12]:
# monetary değerinin satın alma başına ortalama kazanç olarak ifade edilmesi
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

# monetary sıfırdan büyük olanların seçilmesi
cltv_df = cltv_df[cltv_df["monetary"] > 0]
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,310.4400
12347.0000,365,368,7,615.7143
12348.0000,282,359,4,442.6950
12349.0000,0,19,1,1491.7200
12350.0000,0,311,1,331.4600


In [13]:
# BGNBD için recency ve T'nin haftalık cinsten ifade edilmesi
cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

# frequency'nin 1'den büyük olması gerekmektedir.
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [14]:
##############################################################
# 2. BG-NBD Modelinin Kurulması
##############################################################

bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.40, b: 2.49, r: 2.18>

In [15]:
################################################################
# 6 ay içinde en çok satın alma beklediğimiz 10 müşteri kimdir?
################################################################

bgf.conditional_expected_number_of_purchases_up_to_time(6*4,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency'],
                                                        cltv_df['T']).sort_values(ascending=False).head(10)

Customer ID
12748.0000   76.9096
14911.0000   73.6468
17841.0000   45.7014
13089.0000   36.2087
14606.0000   34.4821
15311.0000   33.7688
12971.0000   31.9610
14646.0000   28.7726
13408.0000   23.2283
18102.0000   22.8076
dtype: float64

In [16]:
cltv_df["expected_purc_1_week"] = bgf.predict(6*4,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_week
Customer ID,,,,,
12347.0000,52.1429,52.5714,7,615.7143,3.3239
12348.0000,40.2857,51.2857,4,442.6950,2.1618
12352.0000,37.1429,42.4286,8,219.5425,4.2776
12356.0000,43.1429,46.5714,3,937.1433,2.0218
12358.0000,21.2857,21.5714,2,575.2100,2.8279


In [17]:
##############################################################
# 3. GAMMA-GAMMA Modelinin Kurulması
##############################################################

ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

<lifetimes.GammaGammaFitter: fitted with 2845 subjects, p: 3.79, q: 0.34, v: 3.73>

In [18]:
ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).sort_values(ascending=False).head(10)

Customer ID
12415.0000   5772.1772
12590.0000   5029.4099
12435.0000   4288.9357
12409.0000   3918.8079
14088.0000   3917.1286
18102.0000   3870.9967
12753.0000   3678.5761
14646.0000   3605.3092
15749.0000   3216.0483
14096.0000   3196.4354
dtype: float64

In [19]:
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

In [20]:
cltv_df.sort_values("expected_average_profit", ascending=False).head()

,recency,T,frequency,monetary,expected_purc_1_week,expected_average_profit
Customer ID,,,,,,
12415.0000,44.7143,48.2857,21,5724.3026,8.9256,5772.1772
12590.0000,0.0000,30.2857,2,4591.1725,0.2686,5029.4099
12435.0000,26.8571,38.2857,2,3914.9450,1.7842,4288.9357
12409.0000,14.7143,26.1429,3,3690.8900,2.7280,3918.8079
14088.0000,44.5714,46.1429,13,3864.5546,6.1146,3917.1286


In [21]:
##############################################################
# 4. BG-NBD ve GG modeli ile CLTV'nin hesaplanması.
##############################################################

cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=3,  # 3 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv.head()

Customer ID
12347.0000   1128.4614
12348.0000    538.7980
12352.0000    517.5148
12356.0000   1083.0593
12358.0000    966.6801
Name: clv, dtype: float64

In [22]:
cltv = cltv.reset_index()
cltv.sort_values(by="clv", ascending=False).head()

,Customer ID,clv
1122,14646.0000,55722.4203
2761,18102.0000,47415.6701
843,14096.0000,29066.0386
36,12415.0000,27686.5223
1257,14911.0000,27379.3414


In [23]:
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_average_profit,clv
1122,14646.0000,50.4286,50.7143,74,3596.8044,28.7726,3605.3092,55722.4203
2761,18102.0000,52.2857,52.5714,60,3859.7391,22.8076,3870.9967,47415.6701
843,14096.0000,13.8571,14.5714,17,3163.5882,16.7810,3196.4354,29066.0386
36,12415.0000,44.7143,48.2857,21,5724.3026,8.9256,5772.1772,27686.5223
1257,14911.0000,53.1429,53.4286,201,691.7101,73.6468,692.3264,27379.3414
2458,17450.0000,51.2857,52.5714,46,2863.2749,17.5998,2874.1985,27167.7406
874,14156.0000,51.5714,53.1429,55,2104.0267,20.6664,2110.7541,23425.8613
2487,17511.0000,52.8571,53.4286,31,2933.9431,11.9829,2950.5798,18987.6767
2075,16684.0000,50.4286,51.2857,28,2209.9691,11.2515,2223.8847,13441.1749
650,13694.0000,52.7143,53.4286,50,1275.7005,18.8612,1280.2182,12966.9410


In [24]:
# CLTV'nin Standartlaştırılması
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_final[["clv"]])
cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])  # 0-1 arasına almak için

In [25]:
# Sıralayalım:
cltv_final.sort_values(by="scaled_clv", ascending=False).head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_average_profit,clv,scaled_clv
1122,14646.0000,50.4286,50.7143,74,3596.8044,28.7726,3605.3092,55722.4203,1.0000
2761,18102.0000,52.2857,52.5714,60,3859.7391,22.8076,3870.9967,47415.6701,0.8509
843,14096.0000,13.8571,14.5714,17,3163.5882,16.7810,3196.4354,29066.0386,0.5216
36,12415.0000,44.7143,48.2857,21,5724.3026,8.9256,5772.1772,27686.5223,0.4969
1257,14911.0000,53.1429,53.4286,201,691.7101,73.6468,692.3264,27379.3414,0.4914


In [26]:
##############################################################
# 5. CLTV'ye Göre Segmentlerin Oluşturulması
##############################################################

# Müşterileri 4 gruba ayıralım:
cltv_final["segment"] = pd.qcut(cltv_final["scaled_clv"], 4, labels=["D", "C", "B", "A"])
cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_average_profit,clv,scaled_clv,segment
0,12347.0000,52.1429,52.5714,7,615.7143,3.3239,631.9120,1128.4614,0.0203,A
1,12348.0000,40.2857,51.2857,4,442.6950,2.1618,463.7455,538.7980,0.0097,B
2,12352.0000,37.1429,42.4286,8,219.5425,4.2776,224.8867,517.5148,0.0093,B
3,12356.0000,43.1429,46.5714,3,937.1433,2.0218,995.9977,1083.0593,0.0194,A
4,12358.0000,21.2857,21.5714,2,575.2100,2.8279,631.9010,966.6801,0.0173,A


In [27]:
cltv_final.sort_values(by="scaled_clv", ascending=False).head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_average_profit,clv,scaled_clv,segment
1122,14646.0000,50.4286,50.7143,74,3596.8044,28.7726,3605.3092,55722.4203,1.0000,A
2761,18102.0000,52.2857,52.5714,60,3859.7391,22.8076,3870.9967,47415.6701,0.8509,A
843,14096.0000,13.8571,14.5714,17,3163.5882,16.7810,3196.4354,29066.0386,0.5216,A
36,12415.0000,44.7143,48.2857,21,5724.3026,8.9256,5772.1772,27686.5223,0.4969,A
1257,14911.0000,53.1429,53.4286,201,691.7101,73.6468,692.3264,27379.3414,0.4914,A


In [28]:
# Segmentleri betimleyelim:
cltv_final.groupby("segment").agg{"count", "mean", "sum"})

Customer ID                     recency                        T               frequency                  monetary                expected_purc_1_week              expected_average_profit                         clv                 scaled_clv             
                  sum       mean count        sum    mean count        sum    mean count       sum    mean count         sum     mean count                  sum   mean count                     sum     mean count          sum      mean count        sum   mean count
segment                                                                                                                                                                                                                                                                  
D       11077046.0000 15557.6489   712 15715.5714 22.0724   712 28816.2857 40.4723   712      2184  3.0674   712 130948.6756 183.9167   712            1180.3299 1.6578   712             141959.4892 199.3813   712  102009.4096  143.2716   712     1.8307 0.0026   712
C       10885739.0000 15310.4627   711 21807.1429 30.6711   711 27091.7143 38.1037   711      2911  4.0942   711 193165.2739 271.6811   711            1998.5369 2.8109   711             206172.9514 289.9760   711  270726.9237  380.7692   711     4.8585 0.0068   711
B       10915854.0000 15352.8186   711 20985.0000 29.5148   711 24751.1429 34.8117   711      3869  5.4416   711 265517.6312 373.4425   711            2684.6171 3.7758   711             280058.6456 393.8940   711  489367.4307  688.2805   711     8.7822 0.0124   711
A       10627572.0000 14947.3586   711 22333.1429 31.4109   711 24518.1429 34.4840   711      8078 11.3615   711 469109.4490 659.7883   711            4530.8313 6.3725   711             487624.0925 685.8285   711 1580168.0736 2222.4586   711    28.3579 0.0399   711

In [29]:
cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_average_profit,clv,scaled_clv,segment
0,12347.0000,52.1429,52.5714,7,615.7143,3.3239,631.9120,1128.4614,0.0203,A
1,12348.0000,40.2857,51.2857,4,442.6950,2.1618,463.7455,538.7980,0.0097,B
2,12352.0000,37.1429,42.4286,8,219.5425,4.2776,224.8867,517.5148,0.0093,B
3,12356.0000,43.1429,46.5714,3,937.1433,2.0218,995.9977,1083.0593,0.0194,A
4,12358.0000,21.2857,21.5714,2,575.2100,2.8279,631.9010,966.6801,0.0173,A


# Görev-2 

In [30]:
def create_cltv_p(dataframe, month=3):

    # 1. Veri Ön İşleme
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]

    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")

    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    today_date = dt.datetime(2011, 12, 11)
    cltv_df = dataframe.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                                    lambda date: (today_date - date.min()).days],
                                                    'Invoice': lambda num: num.nunique(),
                                                    'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

    cltv_df.columns = cltv_df.columns.droplevel(0)
    cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
    cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
    cltv_df = cltv_df[cltv_df["monetary"] > 0]
    cltv_df["recency"] = cltv_df["recency"] / 7
    cltv_df["T"] = cltv_df["T"] / 7
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

    # 2. BG-NBD Modelinin Kurulması
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T'])

    cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                                  cltv_df['frequency'],
                                                  cltv_df['recency'],
                                                  cltv_df['T'])

    cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                                   cltv_df['frequency'],
                                                   cltv_df['recency'],
                                                   cltv_df['T'])

    # 3. GAMMA-GAMMA Modelinin Kurulması
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary'])
    cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                                 cltv_df['monetary'])

    # 4. BG-NBD ve GG modeli ile CLTV'nin hesaplanması.
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency'],
                                       cltv_df['T'],
                                       cltv_df['monetary'],
                                       time=month,  # 3 aylık
                                       freq="W",  # T'nin frekans bilgisi.
                                       discount_rate=0.01)

    cltv = cltv.reset_index()
    cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(cltv_final[["clv"]])
    cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])

    cltv_final["segment"] = pd.qcut(cltv_final["scaled_clv"], 4, labels=["D", "C", "B", "A"])

    return cltv_final

In [31]:
df = retail_mysql_df.copy()

In [32]:
create_cltv_p(df, month=1).sort_values(by="scaled_clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv,segment
1122,14646.0000,50.4286,50.7143,74,3596.8044,1.2225,4.8764,3605.3092,18902.0502,1.0000,A
2761,18102.0000,52.2857,52.5714,60,3859.7391,0.9686,3.8639,3870.9967,16081.1848,0.8508,A
843,14096.0000,13.8571,14.5714,17,3163.5882,0.7288,2.8960,3196.4354,9948.4352,0.5263,A
36,12415.0000,44.7143,48.2857,21,5724.3026,0.3796,1.5140,5772.1772,9395.5355,0.4971,A
1257,14911.0000,53.1429,53.4286,201,691.7101,3.1266,12.4731,692.3264,9284.5720,0.4912,A
2458,17450.0000,51.2857,52.5714,46,2863.2749,0.7475,2.9817,2874.1985,9214.1944,0.4875,A
874,14156.0000,51.5714,53.1429,55,2104.0267,0.8776,3.5007,2110.7541,7944.5251,0.4203,A
2487,17511.0000,52.8571,53.4286,31,2933.9431,0.5089,2.0299,2950.5798,6439.4954,0.3407,A
2075,16684.0000,50.4286,51.2857,28,2209.9691,0.4781,1.9070,2223.8847,4559.5513,0.2412,A
1754,16000.0000,0.0000,0.4286,3,2335.1200,0.4221,1.6644,2479.8017,4432.2415,0.2345,A


In [33]:
create_cltv_p(df, month=12).sort_values(by="scaled_clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv,segment
1122,14646.0000,50.4286,50.7143,74,3596.8044,1.2225,4.8764,3605.3092,207476.9756,1.0000,A
2761,18102.0000,52.2857,52.5714,60,3859.7391,0.9686,3.8639,3870.9967,176649.3421,0.8514,A
843,14096.0000,13.8571,14.5714,17,3163.5882,0.7288,2.8960,3196.4354,105762.2913,0.5098,A
36,12415.0000,44.7143,48.2857,21,5724.3026,0.3796,1.5140,5772.1772,102964.3270,0.4963,A
1257,14911.0000,53.1429,53.4286,201,691.7101,3.1266,12.4731,692.3264,102044.0180,0.4918,A
2458,17450.0000,51.2857,52.5714,46,2863.2749,0.7475,2.9817,2874.1985,101209.2398,0.4878,A
874,14156.0000,51.5714,53.1429,55,2104.0267,0.8776,3.5007,2110.7541,87288.6755,0.4207,A
2487,17511.0000,52.8571,53.4286,31,2933.9431,0.5089,2.0299,2950.5798,70747.4026,0.3410,A
2075,16684.0000,50.4286,51.2857,28,2209.9691,0.4781,1.9070,2223.8847,50044.7602,0.2412,A
650,13694.0000,52.7143,53.4286,50,1275.7005,0.8009,3.1948,1280.2182,48320.4246,0.2329,A


Aralarında çok büyük ölçekli farklar yok.

## Görev 3:

#### 2010-2011 UK müşterileri için 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba(segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.

In [41]:
df = retail_mysql_df.copy()

In [43]:
cltv_final = create_cltv_p(df, month=6).sort_values(by="scaled_clv", ascending=False).head(10)

## Görev 4:

#### Veri tabanına kayıt gönderme

In [45]:
pd.read_sql_query("show databases", conn)

,Database
0,information_schema
1,synan_dsmlbc_db
2,synan_dsmlbc_group_1
3,synan_dsmlbc_group_2
4,synan_dsmlbc_group_3
5,synan_dsmlbc_group_4
6,synan_dsmlbc_group_5
7,synan_dsmlbc_group_6
8,synan_dsmlbc_group_7
9,synan_dsmlbc_group_8


In [38]:
pd.read_sql_query("show tables", conn)

,Tables_in_synan_dsmlbc_db
0,cltv_prediction
1,cltv_prediction_dsmlbc6
2,cltv_prediction_dsmlbc7
3,cltv_prediction_mvk
4,emin_serkan_erdonmez
5,emre_topalgokceli
6,onder_ekrem_tanrikulu_cltv
7,online_retail_2009_2010
8,online_retail_2010_2011
9,online_retail_all


In [46]:
cltv_final.to_sql(name='eyyup_demirkiran', con=conn, if_exists='replace', index=False)